Problem Statement. <br/>

This problem is an interactive problem new to the LeetCode platform. <br/>
We are given a word list of unique words, each word is 6 letters long, and one word in this list is chosen as secret. <br/>
You may call master.guess(word) to guess a word.  The guessed word should have type string and must be from the original list with 6 lowercase letters. <br/>
This function returns an integer type, representing the number of exact matches (value and position) of your guess to the secret word.  Also, if your guess is not in the given wordlist, it will return -1 instead. <br/>
For each test case, you have 10 guesses to guess the word. At the end of any number of calls, if you have made 10 or less calls to master.guess and at least one of these guesses was the secret, you pass the testcase. <br/>
Besides the example test case below, there will be 5 additional test cases, each with 100 words in the word list.  The letters of each word in those testcases were chosen independently at random from 'a' to 'z', such that every word in the given word lists is unique. <br/>

Example 1:
Input: secret = "acckzz", wordlist = ["acckzz","ccbazz","eiowzz","abcczz"] <br/>

Explanation: <br/>
master.guess("aaaaaa") returns -1, because "aaaaaa" is not in wordlist. <br/>
master.guess("acckzz") returns 6, because "acckzz" is secret and has all 6 matches. <br/>
master.guess("ccbazz") returns 3, because "ccbazz" has 3 matches. <br/>
master.guess("eiowzz") returns 2, because "eiowzz" has 2 matches. <br/>
master.guess("abcczz") returns 4, because "abcczz" has 4 matches. <br/>

We made 5 calls to master.guess and one of them was the secret, so we pass the test case. <br/>

Note:  Any solutions that attempt to circumvent the judge will result in disqualification.

# Minimax with Heuristic - O(N ^ 2 * logN) runtime, O(N ^ 2) space, where N is the number of words

In [1]:
from typing import List

class Solution:
    def findSecretWord(self, wordlist: List[str], master: 'Master') -> None:
        
        N = len(wordlist)
        self.H = [[sum(a==b for a,b in zip(wordlist[i], wordlist[j]))
                   for j in range(N)] for i in range(N)]

        possible, path = range(N), ()
        while possible:
            guess = self.solve(possible, path)
            matches = master.guess(wordlist[guess])
            if matches == len(wordlist[0]): return
            possible = [j for j in possible if self.H[guess][j] == matches]
            path = path + (guess,)

    def solve(self, possible, path = ()):
        if len(possible) <= 2: return possible[0]

        ansgrp, ansguess = possible, None
        for guess, row in enumerate(self.H):
            if guess not in path:
                groups = [[] for _ in range(7)]
                for j in possible:
                    if j != guess:
                        groups[row[j]].append(j)
                maxgroup = max(groups, key = len)
                if len(maxgroup) < len(ansgrp):
                    ansgrp, ansguess = maxgroup, guess

        return ansguess

# Elimination - O(N * logN) runtime, O(1) space

In [3]:
from typing import List

class Solution:
    def findSecretWord(self, wordlist: List[str], master: 'Master') -> None:
        
        def countMatches(w1, w2):
            return sum(1 for c1, c2 in zip(w1, w2) if c1 == c2)
        
        def wordScore(char_pos_to_count):
            return lambda word: sum(char_pos_to_count[(c, i)] for i, c in enumerate(word))
                    
        if len(wordlist) == 1:
            master.guess(wordlist[0])
        
        possible = list(wordlist) # for some reason there's an error in master.guess() if wordlist is directly modified
        
        char_pos_to_count = defaultdict(lambda: 0)

        for word in possible:
            for pos, char in enumerate(word):
                char_pos_to_count[(char, pos)] += 1

        possible.sort(key=wordScore(char_pos_to_count))
        
        for _ in range(10):
            
            if len(possible) == 1:
                master.guess(possible[0])
                return
            
            guessed = possible.pop()
            verdict = master.guess(guessed)
            
            if verdict == 6:
                return
            
            possible = [w for w in possible if countMatches(w, guessed) == verdict]
        
        return

In [ ]:
instance = Solution()
instance.findSecretWord("acckzz", ["acckzz","ccbazz","eiowzz","abcczz"])